In [18]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd

In [37]:
file_path = "C:/Users/Sreenath/Desktop/counterspeech/labeled_data.csv"
df = pd.read_csv(file_path)
print(df.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class   
0           0      3            0                   0        3      2  \
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


In [13]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Load the dataset
df = pd.read_csv("C:/Users/Sreenath/Desktop/counterspeech/labeled_data.csv")

# Remove URLs and emojis
def remove_urls_emojis(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

df['tweet'] = df['tweet'].apply(remove_urls_emojis)

# Convert to lowercase
df['tweet'] = df['tweet'].str.lower()

# Remove stop words
stop_words = set(stopwords.words('english'))
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Handle contractions (if necessary)
# import contractions
# df['tweet'] = df['tweet'].apply(contractions.fix)

# ... Add more cleaning steps based on your specific dataset

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sreenath\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [33]:
import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df['tweet'] = df['tweet'].apply(remove_punctuation)

In [29]:
import re

def remove_urls_emojis(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^\w\s]', '', text)  # Remove emojis and other symbols
    return text

df['tweet'] = df['tweet'].apply(remove_urls_emojis)

In [30]:
df = df.dropna(subset=['tweet'])

In [31]:
df['tweet'] = df['tweet'].fillna('unknown')

In [20]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import string

# Load the dataset
df = pd.read_csv("labeled_data.csv")

# Cleaning function to remove URLs and emojis
def remove_urls_emojis(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^\w\s]', '', text)  # Remove emojis and other symbols
    return text

# Apply cleaning
df['tweet'] = df['tweet'].apply(remove_urls_emojis)

# Drop rows with missing tweet text
df = df.dropna(subset=['tweet'])

# Load stop words and remove them from the tweets
nltk.download('stopwords')  # Download stopwords if not already available
stop_words = set(stopwords.words('english'))
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

# Remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

df['tweet'] = df['tweet'].apply(remove_punctuation)

# Randomly sample 2000 tweets from the cleaned dataset
df_sampled = df.sample(n=2000, random_state=42)

# Save the cleaned and sampled dataset to the same file name provided by you
cleaned_data_file = "cleaned_labeled_data.csv"
df_sampled.to_csv(cleaned_data_file, index=False)

print(f"Cleaned and reduced dataset saved to {cleaned_data_file}")


Cleaned and reduced dataset saved to cleaned_labeled_data.csv


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sreenath\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('C:/Users/Sreenath/Desktop/counterspeech/cleaned_labeled_data.csv')

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['tweet'])

X_train, X_test = train_test_split(X, test_size=0.5, train_size=0.5, random_state=42)

In [21]:
df = pd.read_csv('cleaned_labeled_data.csv')
print(df)

      Unnamed: 0  count  hate_speech  offensive_language  neither  class   
0           2326      3            0                   3        0      1  \
1          16283      3            0                   3        0      1   
2          19362      3            0                   1        2      2   
3          16780      3            0                   3        0      1   
4          13654      3            1                   2        0      1   
...          ...    ...          ...                 ...      ...    ...   
1995       10888      3            2                   1        0      0   
1996       15054      3            0                   3        0      1   
1997       18725      3            0                   3        0      1   
1998        7146      3            1                   2        0      1   
1999       16496      6            0                   5        1      1   

                                                  tweet  
0                     934 861

In [27]:
import pandas as pd
import torch
from transformers import RobertaModel, RobertaTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn

# Load your dataset
df = pd.read_csv('C:/Users/Sreenath/Desktop/counterspeech/cleaned_labeled_data.csv')

# Create dummy float labels (e.g., assign all tweets a float value '0.0')
df['label'] = 0.0  # Dummy float label; replace this with actual float label logic if needed

# Splitting the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['label'], test_size=0.3, random_state=42)

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')  # Using base RoBERTa model

# Tokenize the text data
train_encodings = tokenizer(list(X_train), padding=True, truncation=True, return_tensors='pt')
test_encodings = tokenizer(list(X_test), padding=True, truncation=True, return_tensors='pt')

# Convert the labels to PyTorch tensors with float type
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float)  # Float labels for regression
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float)

# Create PyTorch DataLoader for training data
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define a regression head on top of RoBERTa
class RobertaRegressionHead(nn.Module):
    def __init__(self):
        super(RobertaRegressionHead, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.regressor = nn.Linear(self.roberta.config.hidden_size, 1)  # Single output for regression

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids, attention_mask)
        hidden_state = outputs.last_hidden_state[:, 0, :]  # Use [CLS] token's representation
        regression_output = self.regressor(hidden_state)
        return regression_output

model = RobertaRegressionHead()

# Set up optimizer, loss function (MSE for regression), and learning rate scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.MSELoss()  # Mean Squared Error for regression
epochs = 1
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask)
        outputs = outputs.squeeze()  # Since we have a single regression output per sample

        # Compute loss
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    print(f'Epoch {epoch+1}/{epochs}: Loss = {total_loss / len(train_loader)}')

# Save the trained model
model_save_path = 'C:/Users/Sreenath/Desktop/counterspeech/roberta_regression'
model.roberta.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Load the trained model for inference
loaded_model = RobertaRegressionHead()
loaded_model.roberta.from_pretrained(model_save_path)
print(loaded_model)


c:\Users\Sreenath\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

Epoch 1/1: Loss = 0.0018991599574126833


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaRegressionHead(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

In [31]:
# ... rest of your code ...

print(f"Number of tweets trained: {len(train_loader.dataset)}")

Number of tweets trained: 1400


In [42]:
df.to_csv('C:/Users/Sreenath/Desktop/counterspeech/trained_data.csv', index=False)

In [46]:
import pandas as pd
import torch
from transformers import RobertaForCausalLM, RobertaTokenizer, pipeline

# Step 1: Load Your Dataset
df = pd.read_csv('C:/Users/Sreenath/Desktop/counterspeech/trained_data.csv')

# Step 2: Preprocess the Data (if necessary)
# Assuming the tweets are in a column named 'tweet'
tweets = df['tweet'].tolist()

# Step 3: Load a Pre-trained RoBERTa Model for Causal Language Modeling
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForCausalLM.from_pretrained('roberta-base')  # Use a model with causal LM head
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Step 4: Generate Counter-Speech
counter_speech = []
for tweet in tweets:
    generated = generator(tweet, max_new_tokens=50, num_return_sequences=1)  # Generate 50 new tokens
    counter_speech.append(generated[0]['generated_text'])

# Step 5: Save the Results
df['counter_speech'] = counter_speech
df.to_csv('C:/Users/Sreenath/Desktop/counterspeech/hate_speech_data.csv', index=False)


c:\Users\Sreenath\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the trained dataset
dataset_path = 'C:/Users/Sreenath/Desktop/counterspeech/trained_data.csv'
data = pd.read_csv(dataset_path)

# Load the tokenizer and model
model_name = 'roberta-base'  # Replace with your actual model name if different
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Define the input prompt
prompt_text = "Your input prompt here."  # Replace with your actual prompt
input_ids = tokenizer.encode(prompt_text, return_tensors='pt')

# Generation parameters
args = {
    'length': 50,  # Maximum number of tokens to generate
    'temperature': 0.7,  # Controls randomness
    'k': 50,  # Limits the next possible words to the top-k options
    'p': 0.95,  # Nucleus sampling parameter
    'repetition_penalty': 1.2,  # Penalizes repetition
    'num_return_sequences': 1,  # Number of sequences to generate
}

# If you are performing regression, you'll typically want to use the model's forward method
with torch.no_grad():  # Disable gradient calculation
    outputs = model(input_ids=input_ids)
    logits = outputs.logits  # Get the logits from the model output

# Process the logits according to your regression task
predictions = logits.squeeze().numpy()  # Convert to numpy array for further processing

# Output the predictions
print("HS:", prompt_text)  # HS stands for "Human Speech" or input prompt
print("Predictions:", predictions)  # Display the regression predictions


c:\Users\Sreenath\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


HS: Your input prompt here.
Predictions: [0.14271286 0.12606551]


In [7]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the trained dataset
dataset_path = 'C:/Users/Sreenath/Desktop/counterspeech/trained_data.csv'
data = pd.read_csv(dataset_path)

# Load the tokenizer and model
model_name = 'roberta-base'  # Replace with your actual model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Prepare a list to hold predictions
predictions_list = []

# Iterate through each prompt in the dataset
for index, row in data.iterrows():
    prompt_text = row['tweet']  # Replace 'your_prompt_column' with the actual column name
    input_ids = tokenizer.encode(prompt_text, return_tensors='pt')

    # Generate predictions
    with torch.no_grad():  # Disable gradient calculation
        outputs = model(input_ids=input_ids)
        logits = outputs.logits  # Get the logits from the model output

    # Process the logits for regression task (or classification if needed)
    predictions = logits.squeeze().numpy()  # Convert to numpy array

    # Append the results to the predictions list
    predictions_list.append({
        'Prompt': prompt_text,
        'Prediction': predictions.tolist()  # Convert numpy array to list for CSV compatibility
    })

# Create a DataFrame from the predictions list
predictions_df = pd.DataFrame(predictions_list)

# Save the predictions to a new CSV file
output_path = 'C:/Users/Sreenath/Desktop/counterspeech/predictions.csv'  # Adjust path as needed
predictions_df.to_csv(output_path, index=False)

print("Predictions saved to:", output_path)


c:\Users\Sreenath\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predictions saved to: C:/Users/Sreenath/Desktop/counterspeech/predictions.csv


In [9]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM

# Load the trained dataset
dataset_path = 'C:/Users/Sreenath/Desktop/counterspeech/trained_data.csv'
data = pd.read_csv(dataset_path)

# Load the classifier model
classifier_model_name = 'roberta-base'  # Your classification model
classifier_tokenizer = AutoTokenizer.from_pretrained(classifier_model_name)
classifier_model = AutoModelForSequenceClassification.from_pretrained(classifier_model_name)

# Load the text generation model
generator_model_name = 'gpt2'  # Change to your text generation model if needed
generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
generator_model = AutoModelForCausalLM.from_pretrained(generator_model_name)

# Prepare a list to hold results
results_list = []

# Define a threshold for negative sentiment (example value)
threshold = 0.0  # Adjust based on your needs

# Iterate through each prompt in the dataset
for index, row in data.iterrows():
    prompt_text = row['tweet']  # Replace with actual column name
    input_ids = classifier_tokenizer.encode(prompt_text, return_tensors='pt')

    # Generate predictions
    with torch.no_grad():
        outputs = classifier_model(input_ids=input_ids)
        logits = outputs.logits.squeeze().numpy()  # Get logits

    # Interpret logits (example: for regression)
    prediction = logits[0]  # Assuming single output for simplicity

    # Generate counter text if needed
    if prediction < threshold:  # Adjust condition as necessary
        # Generate counter text using the generator model
        gen_input_ids = generator_tokenizer.encode(f"Counter response to: {prompt_text}", return_tensors='pt')
        output_sequences = generator_model.generate(
            input_ids=gen_input_ids,
            max_length=50,
            temperature=0.7,
            num_return_sequences=1,
            do_sample=True
        )
        counter_text = generator_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    else:
        counter_text = "No counter response needed."

    # Store the results
    results_list.append({
        'Prompt': prompt_text,
        'Prediction': prediction,
        'Counter Text': counter_text
    })

# Create a DataFrame from the results list
results_df = pd.DataFrame(results_list)

# Save the results to a new CSV file
output_path = 'C:/Users/Sreenath/Desktop/counterspeech/counter_predictions.csv'  # Adjust path as needed
results_df.to_csv(output_path, index=False)

print("Counter predictions saved to:", output_path)


c:\Users\Sreenath\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Counter predictions saved to: C:/Users/Sreenath/Desktop/counterspeech/counter_predictions.csv


In [11]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the trained dataset
dataset_path = "C:/Users/Sreenath/Desktop/counterspeech/hate_speech_data.csv"
data = pd.read_csv(dataset_path)

# Load the pre-trained GPT model and tokenizer
model_name = "gpt2"  # You can choose a larger model if needed
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set pad_token_id
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to be the same as the end of sentence token

# Function to generate a response
def generate_response(prompt, max_new_tokens=50):
    # Encode the input prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt")

    # Create an attention mask
    attention_mask = torch.ones(inputs.shape, device=inputs.device)  # all ones for attention mask

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,  # Generate a specified number of new tokens
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id
        )
    
    # Decode the generated response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# List to hold the generated responses
responses = []

# Generate responses for each entry in the dataset
for index, row in data.iterrows():
    prompt = row['counter_speech']  # Replace with the name of the column you want to use for prompts
    response = generate_response(prompt)
    responses.append(response)

# Create a new DataFrame to store the responses
response_df = pd.DataFrame({'prompt': data['counter_speech'], 'response': responses})  # Make sure to use the correct column name

# Save the responses to a new CSV file
response_df.to_csv("C:/Users/Sreenath/Desktop/counterspeech/generated_responses.csv", index=False)

print("Responses generated and saved to 'generated_responses.csv'")


c:\Users\Sreenath\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Responses generated and saved to 'generated_responses.csv'


In [12]:
print(f"Responses generated so far: {index + 1} out of {len(data)}")

Responses generated so far: 2000 out of 2000


In [19]:
import csv

# Create a CSV writer
with open("C:/Users/Sreenath/Desktop/counterspeech/generated_responses.csv", "w", newline="" , encoding="utf-8") as csvfile:
    fieldnames = ['prompt', 'response']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    # Write the data rows
    for index, row in response_df.iterrows():
        writer.writerow({'prompt': row['prompt'], 'response': row['response']})

In [22]:
import re
import pandas as pd

def separate_text(text):
    """Separates English text and generated text from the given text."""
    # Define regular expressions for English and generated text patterns
    english_pattern = r"[A-Za-z\s]+"
    generated_pattern = r"[^\s]+"  # Matches any non-whitespace characters

    # Find matches for English and generated text
    english_matches = re.findall(english_pattern, text)
    generated_matches = re.findall(generated_pattern, text)

    # Join the matches into separate strings
    english_text = " ".join(english_matches)
    generated_text = " ".join(generated_matches)

    return english_text, generated_text

# Load the dataset
dataset_path = "C:/Users/Sreenath/Desktop/counterspeech/generated_responses.csv"
data = pd.read_csv(dataset_path)

# Separate English and generated text for each response
english_texts = data['response'].apply(lambda x: separate_text(x)[0])
generated_texts = data['response'].apply(lambda x: separate_text(x)[1])

# Create a new DataFrame with only the new fields
new_data = pd.DataFrame({'english_text': english_texts, 'generated_text': generated_texts})

# Create a new file name
new_file_path = "C:/Users/Sreenath/Desktop/counterspeech/separated_responses.csv"

# Save the changes to the new file
new_data.to_csv(new_file_path, index=False)